In [1]:
import sys
sys.path.append('..')


from core.steps import (
    StartStep, 
    EndStep,
    SummarizationStep,
    TranslationStep,
    ValidationStep
)

from core.scenarios import (
    Scenario,
    ScenarioMode
)

from core.models import OpenAIModel

# Manual

In [2]:
openai_token = 'some_token'
openai_model_name = 'gpt-3.5-turbo-16k-0613'

summarize_model = OpenAIModel(openai_model_name, openai_token)
translate_model = OpenAIModel(openai_model_name, openai_token, temperature=0)
validation_model = OpenAIModel(openai_model_name, openai_token, temperature=0)

summarize_step = SummarizationStep(summarize_model)
translate_step = TranslationStep(translate_model)
validation_step = ValidationStep(validation_model)

start_step = StartStep(summarize_model)
end_step = EndStep(summarize_model)
steps = [start_step, summarize_step, translate_step, validation_step, end_step]

In [3]:
scenario_flow = [
    dict(name='start_step', inputs=dict(task='$task')),
    dict(name='summarization_step', inputs=dict(text='$start_step', action='summarize', words_limitation='50')),
    dict(name='translation_step', inputs=dict(text='$summarization_step', from_language='english', to_language='russian', style='formal')),
    dict(name='validation_step', inputs=dict(answer='$translation_step', task='Translate the following text from english to russian', context='$summarization_step')),
    dict(name='end_step', inputs=dict(result='$translation_step'))
]

In [4]:
text = "Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. \
This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application."

In [5]:
scenario = Scenario()
scenario.setup(steps)

scenario_run = scenario.run(text, ScenarioMode.DEFINED, flow=scenario_flow)

print(scenario_run.result)
print(scenario_run.history)

Отправка идентификаторов конечных пользователей в запросах к OpenAI может помочь им отслеживать и обнаруживать злоупотребления, позволяя им давать более конкретную обратную связь, если в вашем приложении обнаружены нарушения политики.
{'task': {'data': 'Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application.', 'id': 0}, 'start_step': {'data': 'Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application.', 'id': 1}, 'summarization_step': {'data': 'Sending end user IDs in requests to OpenAI can help them track and detect abuse, enabling them to give more specific feedback if any policy violations are found in your application.', 'id': 2}, 'translation_step': {'data':

# Auto

In [2]:
openai_token = 'some_token'
openai_model_name = 'gpt-3.5-turbo-16k-0613'

summarize_model = OpenAIModel(openai_model_name, openai_token)
translate_model = OpenAIModel(openai_model_name, openai_token, temperature=0)
validation_model = OpenAIModel(openai_model_name, openai_token, temperature=0)

summarize_step = SummarizationStep(summarize_model)
translate_step = TranslationStep(translate_model)
validation_step = ValidationStep(validation_model)

end_step = EndStep(summarize_model)
steps = [summarize_step, translate_step, end_step, validation_step]

In [3]:
text = "Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. \
This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application."
task = f"Highlight the main idea of the text using no more 10 words and translate main idea into Russian: {text}"

In [4]:
openai_token = 'some_token'
openai_model_name = 'gpt-3.5-turbo-16k-0613'

control_model = OpenAIModel(openai_model_name, openai_token, history_volume=10, temperature=0)

scenario = Scenario()
scenario.setup(steps, needs_control=True, control_model=control_model)

In [5]:
scenario_run = scenario.run(task, ScenarioMode.AUTO)
print(scenario_run.result)
print(scenario_run.history)

Отправка идентификаторов конечных пользователей помогает OpenAI отслеживать и обнаруживать злоупотребления.
{'task': {'data': 'Highlight the main idea of the text using no more 10 words and translate main idea into Russian: Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application.', 'id': 0}, 'summarization_step': {'data': 'Sending end user IDs helps OpenAI track and detect abuse.', 'id': 1}, 'translation_step': {'data': 'Отправка идентификаторов конечных пользователей помогает OpenAI отслеживать и обнаруживать злоупотребления.', 'id': 2}, 'validation_step': {'data': '{"evaluation": "valid", "explanation": ""}', 'id': 3}, 'end_step': {'data': 'Отправка идентификаторов конечных пользователей помогает OpenAI отслеживать и обнаруживать злоупотребления.', 'id': 4}}
